In [1]:
import numpy as np

scores = np.load('results_f1.npy')
print("\nScores:\n", scores.shape)



Scores:
 (4, 10, 10)


In [2]:
mean_scores = np.mean(scores, axis=2).T
print("\nMean scores:\n", mean_scores)


Mean scores:
 [[0.7998645  0.779371   0.80708684 0.73892778]
 [0.56717226 0.55924098 0.60122825 0.54013409]
 [0.9576671  0.94841728 0.95379106 0.93842634]
 [0.86955861 0.88050027 0.88903868 0.90256266]
 [0.5639794  0.46810877 0.53339495 0.49065312]
 [0.68273276 0.69592272 0.71757077 0.69484887]
 [0.87716876 0.80649534 0.78525483 0.7512365 ]
 [0.94086448 0.93560987 0.95056366 0.92063454]
 [0.71745055 0.6316795  0.67984268 0.60466648]
 [0.80979889 0.79918902 0.8254418  0.74551065]]


In [3]:
from sklearn.ensemble import AdaBoostClassifier
from deslib.des.des_knn import DESKNN
from deslib.des.knora_e import KNORAE
from deslib.des.knora_u import KNORAU
from implementedKNORAE import implementedKNORAE

clfs = {
    'AdB' : AdaBoostClassifier(),
    'dkNN': DESKNN(),    
    'knu': KNORAU(),
    'ikne' : implementedKNORAE(),
}

In [4]:
from scipy.stats import rankdata
ranks = []
for ms in mean_scores:
    ranks.append(rankdata(ms).tolist())
ranks = np.array(ranks)
print("\nRanks:\n", ranks)


Ranks:
 [[3. 2. 4. 1.]
 [3. 2. 4. 1.]
 [4. 2. 3. 1.]
 [1. 2. 3. 4.]
 [4. 1. 3. 2.]
 [1. 3. 4. 2.]
 [4. 3. 2. 1.]
 [3. 2. 4. 1.]
 [4. 2. 3. 1.]
 [3. 2. 4. 1.]]


In [5]:
mean_ranks = np.mean(ranks, axis=0)
print("\nMean ranks:\n", mean_ranks)


Mean ranks:
 [3.  2.1 3.4 1.5]


In [6]:
from scipy.stats import ranksums

alfa = .05
w_statistic = np.zeros((len(clfs), len(clfs)))
p_value = np.zeros((len(clfs), len(clfs)))

for i in range(len(clfs)):
    for j in range(len(clfs)):
        w_statistic[i, j], p_value[i, j] = ranksums(ranks.T[i], ranks.T[j])

In [7]:
from tabulate import tabulate

headers = list(clfs.keys())
names_column = np.expand_dims(np.array(list(clfs.keys())), axis=1)
w_statistic_table = np.concatenate((names_column, w_statistic), axis=1)
w_statistic_table = tabulate(w_statistic_table, headers, floatfmt=".2f")
p_value_table = np.concatenate((names_column, p_value), axis=1)
p_value_table = tabulate(p_value_table, headers, floatfmt=".2f")
print("\nw-statistic:\n", w_statistic_table, "\n\np-value:\n", p_value_table)


w-statistic:
         AdB    dkNN    knu    ikne
----  -----  ------  -----  ------
AdB    0.00    2.04  -0.60    2.34
dkNN  -2.04    0.00  -3.06    2.08
knu    0.60    3.06   0.00    3.14
ikne  -2.34   -2.08  -3.14    0.00 

p-value:
         AdB    dkNN    knu    ikne
----  -----  ------  -----  ------
AdB    1.00    0.04   0.55    0.02
dkNN   0.04    1.00   0.00    0.04
knu    0.55    0.00   1.00    0.00
ikne   0.02    0.04   0.00    1.00


In [8]:
advantage = np.zeros((len(clfs), len(clfs)))
advantage[w_statistic > 0] = 1
advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), headers)
print("\nAdvantage:\n", advantage_table)


Advantage:
         AdB    dkNN    knu    ikne
----  -----  ------  -----  ------
AdB       0       1      0       1
dkNN      0       0      0       1
knu       1       1      0       1
ikne      0       0      0       0


In [9]:
significance = np.zeros((len(clfs), len(clfs)))
significance[p_value <= alfa] = 1
significance_table = tabulate(np.concatenate(
    (names_column, significance), axis=1), headers)
print("\nStatistical significance (alpha = 0.05):\n", significance_table)


Statistical significance (alpha = 0.05):
         AdB    dkNN    knu    ikne
----  -----  ------  -----  ------
AdB       0       1      0       1
dkNN      1       0      1       1
knu       0       1      0       1
ikne      1       1      1       0


In [10]:
stat_better = significance * advantage
stat_better_table = tabulate(np.concatenate(
    (names_column, stat_better), axis=1), headers)
print("Statistically significantly better:\n", stat_better_table)

Statistically significantly better:
         AdB    dkNN    knu    ikne
----  -----  ------  -----  ------
AdB       0       1      0       1
dkNN      0       0      0       1
knu       0       1      0       1
ikne      0       0      0       0
